In [3]:
#Part 1: Importing modules for setting up the necessary environment for the following code to execute correctly
#Importing the datetime module from the Python standard library, which provides classes for working with dates and times
from datetime import datetime
from datetime import timedelta
#force the Europe/Zurich timezone
import time, os
os.environ['TZ'] = 'Europe/Zurich' 
import dateutil
#importing requests module for making HTTP requests and the json module for working with JSON data.
import requests
import json
#importing geo-data Nominatim for translating coordinates into the names of the cities
from geopy.geocoders import Nominatim
#import tabulate for creating a table in order to illustrate the output in a nice way (table format)
from tabulate import tabulate



#Part 2: Creating Transport Connections with the 'Connection' Class
#Connection class: a class that holds information about the transport means (e.g. the trains) and the times of a connection
class Connection:
    def __init__(self, destination_x, destination_y, departure, arrival, transport_means):
        '''
        Connection constructor
        :param destination_x: the latitude of the destination
        :param destination_y: the longitude of the destination
        :param departure_timestamp: a timestamp object to indicate the departure time and date on UTC
        :param arrival_timestamp: a datetime object to indicate the arrival time on UTC
        :param transport_means: a list of the transport means of the connection (e.g. ['IC 5'])
        '''
#This if statement is checking the types of the arguments that have been passed into the method
        if (isinstance(destination_x,float) and isinstance(destination_y,float) and
            isinstance(departure,str) and isinstance(arrival,str) and
            isinstance(transport_means,list)):
#When the provided inputs satisfy the aforementioned conditions, the class instance variables are accordingly set with these inputs           
            self.destination_x = destination_x
            self.destination_y = destination_y
            self.transport_means = transport_means
#The codes below take the departure and arrival time in string format, removes the timezone offset from each string, converts the resulting strings to datetime objects, and assign these datetime objects to the object's departure_time and arrival_time attributes         
            self.departure_time = dateutil.parser.parse(departure.split('+')[0])
            self.arrival_time = dateutil.parser.parse(arrival.split('+')[0])
#If the types of the arguments are not as expected it raises an AttributeError
        else:
            raise AttributeError

#The __str__ method defines a human-readable representation of the object displaying the transport means and the departure and arrival times, it can be seen in the first output table on the Traininfo-Row
    def __str__(self):
        return "{}: {} -> {}".format(self.transport_means, self.departure_time, self.arrival_time)

#The methods 'get_unix_departure_time' and 'get_unix_arrival_time' convert the local departure and arrival times into Unix timestamps respectively
    def get_unix_departure_time(self):
        return int((self.departure_time - datetime(1970,1,1)).total_seconds())
    def get_unix_arrival_time(self):    
        return int((self.arrival_time - datetime(1970,1,1)).total_seconds())



#Part 3: Finding Transportation Connections
#Defining a "find_connection" function which finds the first available connection between an origin and a destination at depart_time and _date (timestamp specified below)
def find_connection(origin, destination, departure_date, departure_time):
    url = 'http://transport.opendata.ch/v1/connections'
#Defining the parameters for the API request according to the website of opendata.ch (https://transport.opendata.ch/docs.html)
    params = {}
    params['from'] = origin
    params['to'] = destination
    params['date'] = departure_date
    params['time'] = departure_time
#'r' sends a GET request to the specified url with the parameters (params) specified above
    r = requests.get(url, params = params)
#Extracting the first connection from the response received from the server after making the HTTP request 
    first_conn = r.json()['connections'][0]
#Extracting specific information from the first_conn dictionary, allowing to access details such as the destination coordinates, departure and arrival times, and the means of transportation for the first available connection
    x = first_conn['to']['station']['coordinate']['x']
    y = first_conn['to']['station']['coordinate']['y']
    departure = first_conn['from']['departure']
    arrival = first_conn['to']['arrival']
    transport_means = first_conn['products'] 
    return Connection(x, y, departure, arrival, transport_means)
#Below we specify the timestamp in order to determine the earliest available connection from the present moment
now = datetime.now()
depart_date = now.strftime("%Y-%m-%d")
depart_time = now.strftime("%H:%M")
#Printing the Intro statement and asking the user where he is and where he wants to go
print("Hello! I'm 'Explore Switzerland', your personal Trip Planner. To assist you in organizing your trip, could you please provide answers to the upcoming questions? Let's get started!")
end=input(str("Where do you want to go?"))
start=input(str('Where are you at the moment?'))
#Storing the the user-input inside the find_connection function
last_conn=find_connection(start, end, depart_date, depart_time)



#Part 4: Activities during the stay
#Creating a list of activities which are available on Yelp
activities = ["Bars", "Cafés", "Restaurants"]

#prompt the user to select an activity out of Bars, Cafés and Restaurants
selected_activity = input("Select an activity (Bars, Cafés, Restaurants): ")

#Check if the selected activity is valid
if selected_activity in activities:
    print("Selected activity:", selected_activity)
else:
    print("Invalid activity selected.")

#Defining the function 'find_activities'
def find_activities(latitude, longitude, open_at, radius):
#Unlike the previous OpenData request, accessing Yelp data now requires an api_key:
    api_key = 'OZKjiwrpl5cRuxxx8WYPNYCOgC7TlbolDx_V0TTwoTH6tbWrKW-AsG2uxRioInqrxn7Nk2spjF9i0QzuRf8DG4DOqOxcfQsack070Ey3AQk43NaIRUgaKtBdKy2NYXYx'
#Creating a dictionary "headers" that contains the authorization information required for the API request. It sets the 'Authorization' header with the value 'Bearer' followed by the API key    
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
#Constructing the url for the API request according to Yelp Fusion (https://docs.developer.yelp.com/docs/fusion-intro)    
    url = f'https://api.yelp.com/v3/businesses/search?term={selected_activity}&latitude={latitude}&longitude={longitude}&radius={radius}&open_at={open_at}'
#Making the actual get request with the url and headers created above and the json() method reads the response content and performs the necessary conversion from JSON to a Python dictionary
    r = requests.get(url, headers=headers)
    data = r.json()
#Using open()in order to open the file 'yelp_daten.json' in write mode and then writes the 'data' variable into the file in JSON format with an indentation of 4 spaces for readability
    with open('yelp_daten.json', 'w') as f:
        f.write(json.dumps(data, indent=4))
#Extracting the list of activities from the 'businesses' key in the data dictionary
    activities_list = data['businesses']
    activities = []
#Itterating through the activities_list, extracting the 'name' and 'rating' values from each activity dictionary and appending them as a tuple to the activities list  
    for activity in activities_list:
        name_rating = activity['name'], activity['rating']
        activities.append(name_rating)

#Sorting the activities based on the rating ([1] = index = second position in the tuple). Furthermore, reverse = True means that the activities are sorted in a descending order
    activities.sort(key=lambda x: x[1], reverse=True)

#Extracting the top 5 activities and returning them
    top_5_activities = activities[:5]
    return top_5_activities



#Part 5: Translating User-Inputted City Names to Coordinates Using Nominatim Geocoder and using the above created find_activities function
#In order for the user to input a city name and not the coordinates we need an instance of the Nominatim-Geocoders
geolocator = Nominatim(user_agent="myGeocoder")
#Storing address information with .geocode(str(end)), whereas 'end' is the location where the user wants to go (user-input)
location = geolocator.geocode(str(end))
#Storing the locations latitude and longtitude as well as open_at and radius in variables
latitude = location.latitude
longitude = location.longitude
open_at = int(time.time())
radius = 500
#Execute and store the top_5_activities
top_5_activities = find_activities(latitude, longitude, open_at, radius)
 


#Part 6: Tabular Representation of Information 
#Creating a list for the table data wheras each inner list represent a row in the table
table_data = [
    ["Departure", start],
    ["Destination", end],
    ["Train Info", last_conn],
    ["Selected Activity", selected_activity]]
#Using the tabulate library in order to create a nice table for illustration purposes of the connection data
table = tabulate(table_data, headers=["Field", "Value"], tablefmt="fancy_grid")
print(table)
#Same as above with but now with the activities
activities_table = tabulate(top_5_activities, headers=[f'{selected_activity}', "Rating"], tablefmt="fancy_grid")
print(activities_table)



#Part 7: 
#Defining a "find_hotels" function which uses the before defined latitude and longitude as an input which allows us to enter city names and not coordinates
def find_hotels(latitude, longitude, radius):
#Once again accessing Yelp data with the same API-Key as before
    api_key = 'OZKjiwrpl5cRuxxx8WYPNYCOgC7TlbolDx_V0TTwoTH6tbWrKW-AsG2uxRioInqrxn7Nk2spjF9i0QzuRf8DG4DOqOxcfQsack070Ey3AQk43NaIRUgaKtBdKy2NYXYx'
#Creating a dictionary "headers" that contains the authorization information required for the API request. It sets the 'Authorization' header with the value 'Bearer' followed by the API key    
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
#Constructing the url for the API request according to Yelp Fusion (https://docs.developer.yelp.com/docs/fusion-intro)
#The hotels are being sorted by their rating. This rating function is not strictly sorted by the rating value, but by an adjusted rating value that takes into account the number of ratings, this is to prevent skewing results to businesses with a single review.
    url = f'https://api.yelp.com/v3/businesses/search?term=hotels&latitude={latitude}&longitude={longitude}&radius={radius}&sort_by=rating&limit=5'
#Making the actual get request with the url and headers created above and the json() method reads the response content and performs the necessary conversion from JSON to a Python dictionary
    r = requests.get(url, headers=headers)
    data = r.json()
#Using open()in order to open the file 'yelp_daten.json' in write mode and then writes the 'data' variable into the file in JSON format with an indentation of 4 spaces for readability
    with open('yelp_daten.json', 'w') as f:
        f.write(json.dumps(data, indent=4))
#Extracting the list of hotels from the 'businesses' key in the data dictionary
    hotels_list = data['businesses']
    hotels = []

 #An inner loop that iterates over each key in the keys list tries to retrieve the corresponding value for the current key from the hotel dictionary. 
 #If the key is not found, it assigns the value 'Not Available' to the variable value
    keys = ['name', 'rating', 'price']

    for hotel in hotels_list:
      name_rating = []
      for key in keys:
        try:
          value = hotel[key]
        except KeyError:
          value = 'Not Available'
        name_rating.append(value)

      hotels.append(tuple(name_rating))

    return hotels
 
#Asking the user how long he stays at the destination
duration = int(input('How long are you staying? (in days): '))
#If the duration is only one day the programm will ask how long the user wants to stay at his destination (in hours) and gives back a train connection 
if duration == 1:
    departure_hours = int(input('In how many hours do you want to leave? '))
    now = datetime.now()
    return_time = now + timedelta(hours=departure_hours)
    return_date = return_time.strftime("%Y-%m-%d")
    leave_time = return_time.strftime("%H:%M")
    return_conn = find_connection(end, start, return_date, leave_time)
    print(f"Here are the train connection details for your journey back home in {departure_hours} hours from now: ")
#As above it will be printed in a table format
    table_data = [
        ["Departure", end],
        ["Destination", start],
        ["Train Info", return_conn],]
#Using the tabulate library in order to create a nice table for illustation purposes of the connection data
    table = tabulate(table_data, headers=["Field", "Value"], tablefmt="fancy_grid")
    print(table)

#If the user wants to stay longer than one day it will display Hotels which are generated using the find_hotel function created above
else:
    top_5_hotels = find_hotels(latitude, longitude, radius)

#Using the tabulate library in order to create a nice table for illustation purposes of the different hotels displaying the name of the hotel, the rating and the price level
    hotels_table = tabulate(top_5_hotels, headers=["Hotels", "Rating", "Price"], tablefmt="fancy_grid")
    print(hotels_table)


print("I hope you enjoyed my service. Have a great Journey!")

Hello! I'm 'Explore Switzerland', your personal Trip Planner. To assist you in organizing your trip, could you please provide answers to the upcoming questions? Let's get started!
Where do you want to go?Bern
Where are you at the moment?Basel
Select an activity (Bars, Cafés, Restaurants): Bars
Selected activity: Bars
╒═══════════════════╤══════════════════════════════════════════════════════╕
│ Field             │ Value                                                │
╞═══════════════════╪══════════════════════════════════════════════════════╡
│ Departure         │ Basel                                                │
├───────────────────┼──────────────────────────────────────────────────────┤
│ Destination       │ Bern                                                 │
├───────────────────┼──────────────────────────────────────────────────────┤
│ Train Info        │ ['IC 6']: 2023-05-26 16:56:00 -> 2023-05-26 17:56:00 │
├───────────────────┼────────────────────────────────────────────